In [2]:
from motion import Motion
import numpy as np
import pandas
from statistics import mode
import os

In [3]:
"""""""""""""""""
Update, Observation, Sigma Matrices
"""""""""""""""""
ObsDim = 2
sSqrtSigma = 0.1*np.identity(2*ObsDim) #0.01

oSqrtSigma = 0.5*np.identity(ObsDim) # 0.05

Update = np.identity(2*ObsDim)
for i in range(ObsDim, 2*ObsDim):
    Update[i-ObsDim,i] = 1.0

Obs = np.concatenate((np.identity(ObsDim), np.zeros((ObsDim, ObsDim))), axis=1)

In [4]:
"""""""""""""""
Generate Noise
"""""""""""""""
def noise(Ntype, sqrtSigmaState, sqrtSigmaObs, ObsDim = 2):

        if Ntype == 0: #state noise
            s = np.random.randn(ObsDim*2, 1)
            s = np.dot(sqrtSigmaState,s)
            return s
        if Ntype == 1: #observation noise
            s = np.dot(sqrtSigmaObs,np.random.randn(ObsDim, 1))
            return s

In [5]:
"""""""""""""""
Generate According To Linear Dynamic System
"""""""""""""""

def trajectory(Obs, Update, init, length, sqrtSigmaState, sqrtSigmaObs):
    state = init
    Observs = []
    for i in range(length):
        obs = np.dot(Obs,state) + noise(1, sqrtSigmaState, sqrtSigmaObs)
        state = np.dot(Update,state) + noise(0, sqrtSigmaState, sqrtSigmaObs)
        Observs.append(obs.flatten())
    return Observs

In [6]:
"""""""""""""""""""""
Generate Multiple Trajectories
"""""""""""""""""""""

def generate(Obs, Update, sSqrtSigma, oSqrtSigma, length, size):
    T = []
    for sequence in range(size):
        x = np.random.uniform(-50., 50.)
        y = np.random.uniform(-50., 50.)
        v_x = np.random.uniform(-2., 2.)
        v = np.random.uniform(np.abs(v_x), 2.0)
        v_y = np.sqrt(v**2 - v_x**2) * np.random.choice([-1., 1.])
        init_s = np.array([[x],[y],[v_x],[v_y]])
        T.append(trajectory(Obs, Update, init_s, length, sSqrtSigma, oSqrtSigma))
    return T

In [7]:
"""""""""""""""
Generate Occluded Data
"""""""""""""""

def occlusion(Ts):
    for T in Ts:
        num = np.random.randint(3, 10)
        p = np.random.randint(0, len(T)-num)
        for i in range(num):
            T[p+i] = np.array([[np.inf], [np.inf]])
    return Ts
    

In [8]:
"""""""""""""""""
Check If Two Trajectories Intersect
"""""""""""""""""
def check_intersect(T1, T2):
    intersect = False
    for i in range(len(T1)):
        if np.abs(np.linalg.norm(T1[i] - T2[i])) < 1.0:
            intersect = True
    return intersect

In [9]:
# parameters
min_len = 5
data_size = 100
num_objects = 5

In [ ]:
directory = str(num_objects) + "_obj"
try:
    os.stat(directory)
except:
    os.mkdir(directory) 
example = 0

while example < data_size:
    print ("Sequence " + str(example))
    folder = str(num_objects)+"_obj/csv_" + str(num_objects)+"_"+str(example)
    try:
        os.stat(folder)
    except:
        os.mkdir(folder) 

    """
    Genrate Example Trajectories
    """
    
    
    Ts = generate(Obs, Update, sSqrtSigma, oSqrtSigma, 100, num_objects)
    #c1 = check_intersect(Ts[0], Ts[1])
    #c2 = check_intersect(Ts[0], Ts[2])
    #c3 = check_intersect(Ts[2], Ts[1])
    """
    while not c1 and not c2 and not c3:
        Ts = generate(Obs, Update, sSqrtSigma, oSqrtSigma, 100, num_objects)
        c1 = check_intersect(Ts[0], Ts[1])
        c2 = check_intersect(Ts[0], Ts[2])
        c3 = check_intersect(Ts[2], Ts[1])
    """
    
    """
    .npy Format
    """

    Ob = []
    for time in range(len(Ts[0])):
        obj_t = []
        for obj in range(len(Ts)):
            obj_t.append(Ts[obj][time])
        Ob.append(obj_t)
    Obs_matrix = np.array(Ob)
    np.save(folder+"/"+str(num_objects)+"_obj.npy", Obs_matrix)


    """
    Creating tracklets
    """
    l = len(Ob[0])
    tracklets_active = []
    tracklets = []
    track_label = [] # label which trajectory the tracklet belongs to
    track_label_active = [] 
    for i in range(len(Ob[0])):
        obvs = Ob[0][i]
        track_label_active.append([i])
        tracklets_active.append(([obvs], [0]))
    for time in range(len(Ob) - 1):
        chosen = []
        delete = []
        for i in range(len(tracklets_active)):
            s = tracklets_active[i][0]
            diff = [np.linalg.norm(s[-1] - o) for o in Ob[time+1]]
            diff_s = sorted(diff)
            if diff_s[0]/diff_s[1]<0.3:
                ind = diff.index(diff_s[0] )
                track_label_active[i].append(ind)
                s.append(Ob[time+1][ind])
                chosen.append(ind)
                if time == len(Ob) - 2:
                    tracklets_active[i][1].append(time+1)
            else:
                tracklets_active[i][1].append(time+1)
                tracklets.append(tracklets_active[i])
                track_label.append(track_label_active[i])
                delete.append(i)
        for i in range(len(delete)):
            del tracklets_active[delete[i]-i]
            del track_label_active[delete[i]-i]
        for ind in list(set(range(len(Ob[time+1])))-set(chosen)):
            tracklets_active.append(([Ob[time+1][ind]],[time+1]))
            track_label_active.append([ind])
    tracklets = tracklets + tracklets_active
    track_label = track_label + track_label_active
    output_tracklet = []
    output_label = []
    
    
    for i in range(len(tracklets)):
        track = tracklets[i]
        label = track_label[i]
        if len(track[0]) > min_len:
            output_tracklet.append(track)
            output_label.append(mode(label))
    Tracks = pandas.DataFrame(output_tracklet)
    Tracks.to_csv(folder+"/tracks.csv")
    
    if len(output_label) > num_objects:
        print (output_label)
        # So that the matching is not trivial
        
        example += 1
        
        sequence = []
        for i in range(max(output_label)+1):
            s = []
            for j in range(len(output_label)):
                if output_label[j] == i:
                    s.append(j)
            sequence.append(s)
        print (sequence)
        Sequence = pandas.DataFrame(sequence)
        Sequence.to_csv(folder+"/correct_match.csv")

        """
        Obtain Feasible Tracklet Pairs
        """

        # threshold for the singular value
        index = 0
        for first in range(len(output_tracklet)):
            for second in range(first+1, len(output_tracklet)):
                sim = True
                time1 = output_tracklet[first][1]
                time2 = output_tracklet[second][1]
                if time1 [1] < time2[0]:
                    # first then second
                    alpha1 = output_tracklet[first][0]
                    alpha2 = output_tracklet[second][0]
                    order = [first, second]
                    t1 = time1
                    t2 = time2
                    gap = time2[0] - time1[1]
                elif time1[0] > time2[1]:
                    #second then first
                    alpha1 = output_tracklet[second][0]
                    alpha2 = output_tracklet[first][0]
                    order = [second, first]
                    t1 = time2
                    t2 = time1
                    gap = time1[0] - time2[1]
                else:
                    sim = False

                if sim:
                    index += 1
                    x1 = np.array([alpha1[i][0] for i in range(len(alpha1))])
                    y1 = np.array([alpha1[i][1] for i in range(len(alpha1))])
                    x2 = np.array([alpha2[i][0] for i in range(len(alpha2))])
                    y2 = np.array([alpha2[i][1] for i in range(len(alpha2))])
                    alpha = pandas.DataFrame([x1, y1, t1, x2, y2,t2, [gap], order])
                    alpha.to_csv(folder+"/alpha"+ str(index) +".csv")

In [ ]:
def build_hankel(alpha):
    n = int(np.floor(len(alpha)/2))
    m = len(alpha) - n + 1
    H_x = np.zeros((n, m))
    H_y = np.zeros((n, m))
    for i in range(m):
        for j in range(n):
            H_x[i, j] = alpha[i+j][0]
            H_y[i, j] = alpha[i+j][1]
    return (H_x, H_y)

Hs = [[],[]] 
for track in output_tracklet:
    H_x, H_y = build_hankel(track[0])
    Hs[0].append(H_x)
    Hs[1].append(H_y)

In [69]:
num_obj = 5
folder = "10725_cvs_final_pr/Experiments/"+str(num_obj)+"_obj/csv_"+str(num_obj)+"_30"
#folder = "3_obj_obsolete/csv_3_4"
data = np.load(folder+"/"+str(num_obj)+"_obj.npy")
csv_data = pandas.read_csv(folder + "/tracks.csv")

# scatter points
scat_data = data.reshape(num_obj*100, 2).T

# n tracks
trak = []
for i in range(len(data[0])):
    trak.append([])

for step in data:
    for i in range(len(step)):
        trak[i].append(step[i])
        
trak_x = []
trak_y = []
for trakl in trak:
    trak_x.append(np.array(trakl).T[0])
    trak_y.append(np.array(trakl).T[1])

# tracklets
traklets = []
for i in range(len(csv_data["0"])):
    traklet = []
    for pair in csv_data["0"][i].split("array"):
        pairs = []
        nums = pair.replace('(', ',').replace('[', ',').replace(']', ',').replace(')', ',').split(',')
        flag = False
        for ele in nums:
            try:
                float(ele)
                pairs.append(float(ele))
                flag = True
            except:
                pass
        if flag:
            traklet.append(pairs)
    traklets.append(np.array(traklet).T)

# stitches 
assign = np.genfromtxt(folder + "/Assignment.csv", delimiter=',')
matched = []
trajectories = []
if len(assign.shape) == 1:
    assign = np.array([assign])
for row in range(assign.shape[0]):
    
    m = row
    
    print (m not in matched)
    print (assign)
    if m not in matched and (1. in assign[m]):
        matched.append(m)
        trajectory = [traklets[m]]
        while m < assign.shape[0]:

            m_prev = m
            m = list(assign[m]).index(1)

            if m not in matched:
                matched.append(m)

            y_star = np.genfromtxt(folder + "/y_star_" + str(m_prev) + "_" + str(m)+".csv", delimiter=',').T
            trajectory.append(y_star)
            trajectory.append(traklets[m])
        trajectories.append(trajectory)

for missed in list(set(range(len(traklets))) - set(matched)):
    trajectories.append([traklets[missed]])
print (len(trajectories))


True
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
True
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
True
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
False
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
5


In [75]:
import matplotlib.pyplot as plt
plt.close()
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')
for i in range(len(trak_x)):
    ax1.plot(trak_x[i], trak_y[i])

ax1.set_title("Original Track")
ax1.set_ylabel("y")
ax2.scatter(scat_data[0], scat_data[1], c = "g", marker = "*", s = 0.5)
ax2.set_title("Observations")

for i in range(len(traklets)):
    arry = traklets[i]
    ax3.plot(arry[0], arry[1], label="Tracklet" + str(i))
#ax3.legend()
ax3.set_xlabel("x")
ax3.set_ylabel("y")
ax3.set_title("Heuristic Tracklets")


color = ["xkcd:azure", "orange",  "orchid", "green", "red"]
stitch_color = ["xkcd:blue", "xkcd:orangered","indigo", "xkcd:teal", "xkcd:magenta"]
#color = [ "xkcd:blue","orange", "green" ]
#stitch_color = [ "xkcd:azure", "xkcd:orangered","xkcd:teal"]
for t in range(len(trajectories)):
    print (t)
    trajectory = trajectories[t]
    for p in range(len(trajectory)):
        piece = trajectory[p]
        if p%2 == 0:
            ax4.plot(piece[0], piece[1], color=color[t])
        else:
            ax4.plot(piece[0], piece[1], linestyle='-.', color=stitch_color[t])
ax4.set_xlabel("x")

ax4.set_title("Stitched Tracklets")
#plt.show()
plt.savefig("10725_cvs_final_pr/plots/"+str(num_obj)+"_obj.png")

0
1
2
3
4


In [ ]:
plt.close()
for i in range(len(trak_x)):
    plt.plot(trak_x[i], trak_y[i])

plt.title("Original Track")
plt.savefig("10725_cvs_final_pr/plots/"+str(num_obj)+"_obj_trajectory.png")


In [ ]:
plt.close()
plt.scatter(scat_data[0], scat_data[1], c = "g", marker = "*", s = 0.5)
plt.title("Observations")
plt.savefig("10725_cvs_final_pr/plots/"+str(num_obj)+"_obj_scatter.png")

In [ ]:
plt.close()
for i in range(len(traklets)):
    arry = traklets[i]
    plt.plot(arry[0], arry[1], label="Tracklet" + str(i))
#ax3.legend()
plt.title("Heuristic Tracklets")
plt.savefig("10725_cvs_final_pr/plots/"+str(num_obj)+"_obj_tracklets.png")

In [ ]:
plt.close()
color = ["xkcd:azure", "orange","orchid",  "green", "red"]
stitch_color = ["xkcd:blue", "xkcd:orangered","indigo",  "xkcd:teal", "xkcd:magenta"]
#color = [ "green", "xkcd:blue", "orange"]
#stitch_color = [ "xkcd:teal","xkcd:azure","xkcd:orangered"]
for t in range(len(trajectories)):
    print (t)
    trajectory = trajectories[t]
    for p in range(len(trajectory)):
        piece = trajectory[p]
        if p%2 == 0:
            plt.plot(piece[0], piece[1], color=color[t])
        else:
            plt.plot(piece[0], piece[1], linestyle='-.', color=stitch_color[t])
plt.title("Stitched Tracklets")
plt.savefig("10725_cvs_final_pr/plots/"+str(num_obj)+"_obj_stitched.png")

In [ ]:
assign = pandas.read_csv(folder + "/Assignment.csv")
assign = assign.values
matched = []
trajectories = []
for row in range(assign.shape[0]):
    if any(matched) is not row:
        matched.append(row)
    m = row
    trajectory = [traklets[m]]
    while m < assign.shape[0]:
        m_prev = m
        m = list(assign[m]).index(1)
        y_star = pandas.read_csv(folder + "/y_star_" + str(m_prev) + "_" + str(m)+".csv").values.T
        trajectory.append(y_star)
        trajectory.append(traklets[m])
    trajectories.append(trajectory)
print (trajectories)

In [ ]:
a = [1, 5, 6]
if any(a) is not 2:
    print ("dumb")

In [ ]:
list(assign[0]).index(1)

In [ ]:
traklets[0]

In [ ]:
x = np.array([0, 0, 0, 0, 1])
len(x.shape)